In [126]:
using NCDatasets
using ImageFiltering
using Statistics
using Images
using Missings

###what is the most eloquent way to factor out the middle gate in the calculation? 

In [7]:
Images.meanfinite

meanfinite (generic function with 3 methods)

In [125]:
import Pkg
Pkg.add("Missings")

   Resolving package versions...
    Updating `~/.julia/environments/v1.9/Project.toml`
  [e1d29d7a] + Missings v1.1.0
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


missing

In [131]:
avg_weights

5×5 Matrix{Union{Missing, Float64}}:
 1.0  1.0  1.0       1.0  1.0
 1.0  1.0  1.0       1.0  1.0
 1.0  1.0   missing  1.0  1.0
 1.0  1.0  1.0       1.0  1.0
 1.0  1.0  1.0       1.0  1.0

In [144]:

##Weight matrixes for calculating spatial variables 
##TODO: Add these as default arguments for their respective functions 

USE_GATE_IN_CALC = false 

function get_NCP(data::NCDataset)
    ###Some ternary operator + short circuit trickery here 
    (("NCP" in keys(data)) ? (return(data["NCP"][:,start_scan_idx:1:end_scan_idx]))
                        : ("SQI" in keys(data) ||  error("Could Not Find NCP in dataset")))
    return(data["SQI"][:, start_scan_idx:1:end_scan_idx])
end

##Applies function given by func to the weighted version of the matrix given by var 
##Also applies controls for missing variables to both weights and var 
function _weighted_func(var::AbstractMatrix{}, weights, func)
    
    valid_weights = .!map(ismissing, weights)
    
    
    updated_weights = weights[valid_weights]
    updated_var = var[valid_weights]
    
    valid_idxs = .!map(ismissing, updated_var)
    
    ##Returns 0 when missing, 1 when not 
    return func(updated_var[valid_idxs] .* updated_weights[valid_idxs])
end

###need to consider how to optimize this 

##Calculate the isolation of a given variable 
###These actually don't necessarily need to be functions of their own, could just move them to
###Calls to _weighted_func 
function calc_isolation_param(var::AbstractMatrix{Union{Missing, Float64}}, weights, window)
    
    if size(weights) != window
        error("Weight matrix does not equal window size")
    end
    
    missings = map((x) -> ismissing(x), var)
    iso_array = mapwindow((x) -> _weighted_func(x, weights, sum), missings, window) 
end

##Calculate the isolation of a given variable 
###These actually don't necessarily need to be functions of their own, could just move them to
###Calls to _weighted_func 
function calc_isolation_param(var::AbstractMatrix{Union{Missing, Float32}}, weights, window)
    
    if size(weights) != window
        error("Weight matrix does not equal window size")
    end
    
    missings = map((x) -> ismissing(x), var)
    iso_array = mapwindow((x) -> _weighted_func(x, weights, sum), missings, window) 
end


###pass this to mapwindow, var and weights should have same dimension so can index similarily
###PROBLEM: Initial will still factor into calculation of number of average
function _missing_avg(var::AbstractMatrix{Union{Missing, Float32}}, weights)
    
    valid_idxs = .!map(ismissing, var)
    return(mean((var[valid_idxs] .* weights[valid_idxs])))
end

function _missing_avg(var, weights)
    
    valid_idxs = .!map(ismissing, var)
    return(mean((var[valid_idxs] .* weights[valid_idxs])))
end


function _missing_weights_avg(var, weights) 
    
    valid_weights = .!map(ismissing, weights)
    
    
    updated_weights = weights[valid_weights]
    updated_var = var[valid_weights]
    
    valid_idxs = .!map(ismissing, updated_var)
    return(mean((updated_var[valid_idxs] .* updated_weights[valid_idxs])))
end



##Calculate the windowed standard deviation of a givne variable 
function calc_std(var::AbstractMatrix{Union{Missing, Float64}}, weights, window)
    
    if size(weights) != window
        error("Weight matrix does not equal window size")
    end
    
    mapwindow((x) -> _weighted_func(x, weights, std), var, window)
end 

function calc_avg(var::Matrix{Union{Missing, Float32}}, weights, window)
    
    if size(weights) != window
        error("Weight matrix does not equal window size")
    end
    
    mapwindow((x) -> _missing_weights_avg(x, weights), var, window)
end



##Can be done on a ray-by-ray basis, otherwise the elevation angle will change 
##What is this algorithm? 
function airborne_ht(elevation_angle::Float64, antenna_range::Float64, aircraft_height::Float64)
    ##Initial heights are in meters, convert to km 
    aRange_km, acHeight_km = (antenna_range, aircraft_height) ./ 1000
    
    term1 = aRange_km^2 + EarthRadiusKm^2
    term2 = 2 * aRange_km * EarthRadiusKm * sin(deg2rad(elevation_angle))
    return sqrt(term1 + term2) - EarthRadiusKm + acHeight_km
end 

###How can we optimize this.... if one gate has a probability of ground equal to 1, the rest of 
###The gates further in that range must also have probability equal to 1, correct? 
function prob_groundgate(elevation_angle, antenna_range, aircraft_height, azimuth, max_range)
    
    ###If range of gate is less than altitude, cannot hit ground
    ###If elevation angle is positive, cannot hit ground 
    if (antenna_range > aircraft_height || elevation_angle > 0)
        return 0. 
    end 
    elevation_rad, azimuth_rad = map((x) -> deg2rad(x), (elevation_angle, azimuth))
    
    #Range at which the beam will intersect the ground (Testud et. al 1995 or 1999?)
    Earth_Rad_M = EarthRadiusKm*1000
    ground_intersect = (-(aircraft_height)/sin(elevation_rad))*(1+aircraft_height/(2*Earth_Rad_M* (tan(elevation_rad)^2)))
    
    range_diff = ground_intersect - antenna_range
    
    if (range_diff <= 0)
            return 1.
    end 
    
    gelev = asin(-aircraft_height/antenna_range)
    elevation_offset = elevation_rad - gelev
    
    if elevation_offset < 0
        return 1.
    else
        
        beamaxis = sqrt(elevation_offset * elevation_offset)
        gprob = exp(-0.69314718055995*(beamaxis/beamwidth))
        
        if (gprob>1.0)
            return 1.
        else
            return gprob
        end
    end 
end 

prob_groundgate (generic function with 1 method)

In [11]:

path = "CFRADIALS/cfrad.19950516_221944.169_to_19950516_221946.124_TF-ELDR_AIR.nc"

"CFRADIALS/cfrad.19950516_221944.169_to_19950516_221946.124_TF-ELDR_AIR.nc"

In [12]:
currset = Dataset(path)

Dataset: CFRADIALS/cfrad.19950516_221944.169_to_19950516_221946.124_TF-ELDR_AIR.nc
Group: /

Dimensions
   time = 182
   range = 384
   sweep = 1
   string_length_8 = 8
   string_length_32 = 32
   status_xml_length = 1
   r_calib = 1
   frequency = 4

Variables
  volume_number  
    Attributes:
     long_name            = data_volume_index_number
     units                = 
     _FillValue           = -9999

  platform_type   (32)
    Datatype:    Char (Char)
    Dimensions:  string_length_32
    Attributes:
     long_name            = platform_type
     options              = fixed, vehicle, ship, aircraft_fore, aircraft_aft, aircraft_tail, aircraft_belly, aircraft_roof, aircraft_nose, satellite_orbit, satellite_geostat

  primary_axis   (32)
    Datatype:    Char (Char)
    Dimensions:  string_length_32
    Attributes:
     long_name            = primary_axis_of_rotation
     options              = axis_z, axis_y, axis_x, axis_z_prime, axis_y_prime, axis_x_prime

  status_xml   (1)


In [149]:

iso_weights = allowmissing(ones(7,7))
iso_weights[4,4] = missing
iso_window = (7,7)

avg_weights = allowmissing(ones(5,5))
avg_weights[3,3] = missing
avg_window = (5,5)

(5, 5)

In [160]:
@time calc_isolation_param(currset["DBZ"][:,:], iso_weights, iso_window)

  0.098643 seconds (817.89 k allocations: 143.718 MiB, 16.67% gc time)


384×182 Matrix{Float64}:
 29.0  32.0  37.0  40.0  44.0  44.0  …   0.0   0.0   0.0   0.0   0.0   0.0
 33.0  36.0  39.0  42.0  45.0  45.0      0.0   0.0   0.0   0.0   0.0   0.0
 38.0  40.0  42.0  44.0  46.0  46.0      0.0   0.0   0.0   0.0   0.0   0.0
 43.0  44.0  45.0  46.0  47.0  47.0      0.0   0.0   0.0   0.0   0.0   0.0
 48.0  48.0  48.0  48.0  48.0  48.0      0.0   0.0   0.0   0.0   0.0   0.0
 48.0  48.0  48.0  48.0  48.0  48.0  …   0.0   0.0   0.0   0.0   0.0   0.0
 48.0  48.0  48.0  48.0  48.0  48.0      0.0   0.0   0.0   0.0   0.0   0.0
 48.0  48.0  48.0  48.0  48.0  48.0      0.0   0.0   0.0   0.0   0.0   0.0
 48.0  48.0  48.0  48.0  48.0  48.0      0.0   0.0   0.0   0.0   0.0   0.0
 48.0  48.0  48.0  48.0  48.0  48.0      0.0   0.0   0.0   0.0   0.0   0.0
 48.0  48.0  48.0  48.0  48.0  48.0  …   0.0   0.0   0.0   0.0   0.0   0.0
 48.0  48.0  48.0  48.0  48.0  48.0      0.0   0.0   0.0   0.0   0.0   0.0
 48.0  48.0  48.0  48.0  48.0  48.0      0.0   0.0   0.0   0.0   0.0   0.0


In [162]:
currset["DBZ"][:,:]

384×182 Matrix{Union{Missing, Float32}}:
 -34.99      missing  -34.99      …  -34.99      -34.99      -34.99
    missing  missing     missing       0.75        6.25        9.63
    missing  missing     missing      11.13       10.38        9.88
    missing  missing     missing      11.38       13.13        8.63
    missing  missing     missing       9.75        9.5        11.63
    missing  missing     missing  …    6.75        9.25       11.13
    missing  missing     missing       6.88        7.25        7.38
    missing  missing     missing       5.63        8.25        7.5
    missing  missing     missing       7.38        6.25        5.75
    missing  missing     missing      10.63        8.88        9.63
    missing  missing     missing  …   13.25       13.25       14.5
    missing  missing     missing      16.25       14.25       13.63
    missing  missing     missing      21.13       18.38       14.63
   ⋮                              ⋱                ⋮         
    missing  mi

In [146]:
size(iso_weights) == iso_window

false

In [147]:
iso_window

7×7 Matrix{Float64}:
 1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0
 1.0  1.0  1.0  1.0  1.0  1.0  1.0

In [141]:
@time calc_avg(currset["VV"][:,:], weights, window)

  0.166874 seconds (872.22 k allocations: 84.067 MiB, 52.40% gc time)


384×182 Matrix{Float64}:
   6.0948   11.6508   13.5696   19.8096  …   -5.67     -5.8188   -5.9368
   4.6932   12.5732   11.3      12.9132      -5.4556   -5.6544   -5.762
   4.0172    7.498    11.1072    9.5316      -5.196    -5.3644   -5.5508
   5.9568    5.646     3.4344    4.2072      -4.8352   -4.9856   -5.108
   3.1556    1.8552    0.152    -0.034       -4.4508   -4.5624   -4.6508
  -2.4568   -4.568    -4.762    -1.7936  …   -4.6256   -4.7312   -4.8688
  -8.204    -8.8108   -6.9316   -3.6696      -4.8656   -4.9236   -4.9356
  -8.3512   -7.8476   -6.2368   -4.3836      -5.552    -5.5104   -5.468
  -6.0612   -6.204    -4.6176   -2.7212      -6.5588   -6.5728   -6.5672
  -6.688    -5.3012   -3.9416   -2.4336      -7.628    -7.6088   -7.6636
  -5.3756   -3.3596   -1.3076    0.854   …   -8.2944   -8.2324   -8.2144
   1.0328    1.0544    1.0284    1.1996      -8.6312   -8.6136   -8.6312
   0.9692    1.1188    1.2472    1.4388      -8.5056   -8.5272   -8.5692
   ⋮                         

In [66]:
toy_var = [1. 2. 3.; missing 2. 3.; 3. 3. 1.]

3×3 Matrix{Union{Missing, Float64}}:
 1.0       2.0  3.0
  missing  2.0  3.0
 3.0       3.0  1.0

In [74]:
toy_weights = [1 1 1; 1 missing 1; 1 1 1]

3×3 Matrix{Union{Missing, Int64}}:
 1  1         1
 1   missing  1
 1  1         1

In [116]:
@time _missing_weights_avg(toy_var, toy_weights)

OK  0.000098 seconds (22 allocations: 1.141 KiB)


2.2857142857142856

In [72]:
toy_var

3×3 Matrix{Union{Missing, Float64}}:
 1.0       2.0  3.0
  missing  2.0  3.0
 3.0       3.0  1.0

In [76]:
var[.!(map(ismissing, toy_weights))]

LoadError: MethodError: no method matching getindex(::typeof(var), ::BitMatrix)

In [42]:
mean

mean (generic function with 11 methods)

In [98]:
 
valid_weights = .!map(ismissing, toy_weights)
valid_idxs = .!map(ismissing, toy_var)

updated_weights = toy_weights[valid_weights]
updated_var = toy_var[valid_weights]

8-element Vector{Union{Missing, Float64}}:
 1.0
  missing
 3.0
 2.0
 3.0
 3.0
 3.0
 1.0